In [ ]:
import rasterio  
import rasterio.mask
import geopandas as gps
import os
import time
import shutil
from shapely.geometry import Point, Polygon,box
import numpy as np

In [ ]:
all_files=[]
for root, dirs, files in os.walk(r'J:\4_GSW\occurrence_2010_2019'):
    for file in files:
        # if file.endswith(config.input_image_type)and file.startswith('sample'):
        if file.endswith('.tif')and file.startswith('occurrence'):
             all_files.append((os.path.join(root, file), file))
print(all_files)

out_dir=r'J:\4_GSW\P3\v1_0221\after_GLAKES_mask'

for fullPath, filename in all_files:  
    raw_occurrence_tif=rasterio.open(fullPath)
    out_occurrence_tif=os.path.join(out_dir,filename.replace('.tif','_after_lake_mask.tif'))
    if not os.path.exists(out_occurrence_tif):
        bbox = box(*raw_occurrence_tif.bounds)
        meta=raw_occurrence_tif.meta
        meta.update({"compress": 'lzw'})
        GLAKES_mask = gps.read_file(r'D:\lakemapping\0_auxiliary_data\GLAKES\GLAKES\GLAKES.gdb',layer='GLAKES', bbox=bbox) 
        mask_patch_geojson = GLAKES_mask.geometry.values 

        if len(mask_patch_geojson) > 0:
            modelpredtictions_masked, out_meta = rasterio.mask.mask(raw_occurrence_tif, mask_patch_geojson, all_touched=False, invert=False)
            print(f'Mask finished: {filename}')
            with rasterio.open(out_occurrence_tif, "w", **meta) as prediction_masked:
                prediction_masked.write(modelpredtictions_masked)
    

In [ ]:
print(len(all_files))